<a href="https://colab.research.google.com/github/mojtabaSefidi/Fall-Detection-System/blob/master/Fall_Detection_System_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


Import libraries and frameworks

In [3]:
from sklearn import preprocessing
import tensorflow as tf
from math import sqrt
import pandas as pd
import numpy as np
import glob
import os
window_size = 200
from sklearn.utils import class_weight
import matplotlib.pyplot as plt
import seaborn as sn
sn.set()
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
import random
from itertools import groupby, chain


Get the sisfall and sisfall_enhanced dataset

In [4]:

# !unzip /content/gdrive/MyDrive/Datasets/SisFall_dataset.zip
# !unzip /content/gdrive/MyDrive/Datasets/SisFall_enhanced.zip

# Implementation

In [5]:
def plot_confusion_matrix(confusion_matrix):
    df = pd.DataFrame(confusion_matrix, range(len(confusion_matrix)), range(len(confusion_matrix)))
    plt.figure(figsize=(6,4))
    plt.title('Confusion Matrix')
    sn.set(font_scale=1) # for label size
    sn.heatmap(df, annot=True, annot_kws={"size": 12},fmt='.0f',cmap='Greens') # font size
    plt.show()

Get all addresses

In [6]:
def get_file_name(path, ratio=0.8):
  allfiles = []
  allFolders = sorted(glob.glob(path + "*"))
  for files in allFolders:
    allfiles.append(sorted(glob.glob(files+"/*.txt")))
  if 'desktop.ini' in allfiles:
        allfiles.remove('desktop.ini')

  dataset = np.hstack(allfiles)
  start = dataset[0].rfind('/') + 1
  end = dataset[0][start:].find('_') + start
  dataset = [list(g) for k, g in groupby(dataset, key=lambda x: x[start:end])]
  train = []
  test = []
  for data in dataset:
    if len(data) == 1:
      if random.randint(1,100)>=81:
        test.extend(data)
      else:
        train.extend(data)

    else:
      random.shuffle(data)
      train.extend(data[:int(len(data)*ratio)])
      test.extend(data[int(len(data)*ratio):])

  return train, test


read dataset from address path

In [40]:
class DatasetProcessor():

  def __init__(self,
               train_dataset_saving_path=None,
               test_dataset_saving_path=None
               ):
    self.train_dataset_saving_path = train_dataset_saving_path
    self.test_dataset_saving_path = test_dataset_saving_path

  def __read_data(self, data_path):
    data = pd.read_csv(data_path, header=None)
    data.columns = ['ADXL345_x', 'ADXL345_y', 'ADXL345_z', 'ITG3200_x', 'ITG3200_y', 'ITG3200_z', 'MMA8451Q_x',
                    'MMA8451Q_y', 'MMA8451Q_z']
    data['MMA8451Q_z'] = data['MMA8451Q_z'].map(lambda x: str(x)[:-1])
    for name in data.columns :
      data[name] = data[name].astype(float)
    return data

  def __add_label(self, data_path, merge_feature=False):

    dataset = self.__read_data(data_path)

    if not merge_feature:
      dataset['label'] = self.__get_label(data_path)
      return dataset.to_numpy()

    else:
      new_dataset = pd.DataFrame()
      new_dataset['acc_1'] = dataset.apply(
          lambda row: sqrt((row.ADXL345_x ** 2 + row.ADXL345_y ** 2 + row.ADXL345_z ** 2)), axis=1)
      new_dataset['acc_2'] = dataset.apply(
          lambda row: sqrt((row.MMA8451Q_x ** 2 + row.MMA8451Q_y ** 2 + row.MMA8451Q_z ** 2)), axis=1)
      new_dataset['geo'] = dataset.apply(
          lambda row: sqrt((row.ITG3200_x ** 2 + row.ITG3200_y ** 2 + row.ITG3200_z ** 2)), axis=1)
      new_dataset['label'] = self.__get_label(data_path)

      return np.round(new_dataset.to_numpy(), 2)

  def __get_label(self, data_path):
    label = data_path[54]
    if label =='D':
      return int(0)
    elif label =='F':
      label_path = data_path.replace('dataset', 'enhanced')
      labels = pd.read_csv(label_path, header=None)
      # labels[labels == 2] = 1
      return labels

  def datasets_to_nparray(self, datasets_address_array, outputsize=20000000, column_dimension=10):
    result = np.zeros((outputsize, column_dimension), 'int16')
    first_index = 0
    for address in datasets_address_array:
      feature = self.__add_label(address)
      # print(len(feature), first_index)
      result[first_index : (first_index+len(feature))] = feature
      first_index += len(feature)

    return result[result.sum(axis=1) != 0]

  def windowing2d(self, dataset, window_size=200):
    window = window_size * (dataset.shape[1]-1)
    cut = dataset.shape[0] % window_size
    feature = dataset[:-cut,0:-1]
    label = dataset[:-cut,-1]
    feature = feature.ravel().reshape(feature.size//window,window)
    label = label.reshape(label.size// window_size, window_size)
    label = label.sum(axis=1)
    label[label > 0] = 1
    return feature, label

  def windowing3d(self, dataset, window_size=200):
    n_windows = len(dataset) // window_size
    cut = dataset.shape[0] % window_size
    feature = dataset[:-cut,0:-1]
    label = dataset[:-cut,-1]
    feature = feature.reshape(n_windows, window_size, dataset.shape[1]-1)
    label = label.reshape(n_windows, window_size, 1)
    label = label.sum(axis=1)
    label[label > 0] = 1
    return feature, label

  def normalizer(self, scaler, X_train, X_test):
    X_train = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
    X_test = scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)
    return X_train, X_test

  def dataset_to_tensor(self, window_size, dataset, saving_path):
    features, labels = self.windowing(self.__datasets_to_nparray(dataset), window_size)
    return features, labels
    # return np.savez(saving_path, inputs=features, targets=labels)

  def dataset_loader(self, data_path):
    npz = np.load(data_path)
    return npz["inputs"].astype(np.float), npz["targets"].astype(np.int)

  def downsampling(self, dataset, down_sampleing_factor):
      positive = dataset[dataset['targets']==1]
      negative = dataset[dataset['targets']==0].sample(n=int(len(positive)* down_sampleing_factor))
      return pd.concat([positive, negative], ignore_index=True).sample(frac=1).reset_index(drop=True)

  def generate_class_weight(self, label):
    try:
      return class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(label), y=label)
    except:
      return class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(label.ravel()), y=label.ravel())


In [48]:
%%time

print('1.Split Adresses...')
train, test = get_file_name('/content/gdrive/MyDrive/Datasets/SisFall_dataset/')

dp = DatasetProcessor()

print('2.Extract Features and Labels...')

print('------------------------Train Dataset')
train_dataset = dp.datasets_to_nparray(train[:50])

print('------------------------Test Dataset')
test_dataset = dp.datasets_to_nparray(test[:50])

print('3.Windowing...')
print('------------------------Train Dataset')
X_train, y_train = dp.windowing2d(train_dataset)
print('------------------------Test Dataset')
X_test, y_test = dp.windowing2d(test_dataset)

print('4.Normalizing...')
X_train, X_test = dp.normalizer(StandardScaler(), X_train, X_test)

print('5.Calculate Class Weight...')
weight = dp.generate_class_weight(y_train)


1.Split Adresses...
2.Extract Features and Labels...
------------------------Train Dataset
------------------------Test Dataset
3.Windowing...
------------------------Train Dataset
------------------------Test Dataset
4.Normalizing...
5.Calculate Class Weight...
CPU times: user 2.97 s, sys: 242 ms, total: 3.21 s
Wall time: 16 s


((967, 1800), (967,), (822, 1800), (822,))

# Main

Data Pre-processing

Neural Network Training

In [ ]:
class Models():

  def __init__(self,
               X_train,
               y_train,
               X_test,
               y_test,
               ):

        self.X_train = X_train
        self.y_train = y_train
        self.X_test = X_test
        self.y_test = y_test


  def define_mlp(self, input_size, hidden_layer_size, output_size):

    self.mlp_model = tf.keras.Sequential([
                                tf.keras.layers.Dense(input_size,activation="relu"),
                                tf.keras.layers.Dense(hidden_layer_size, activation="relu"),
                                tf.keras.layers.Dense(hidden_layer_size,activation="relu"),
                                tf.keras.layers.Dense(output_size, activation='sigmoid')
                                ])

    self.mlp_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

  def train_mlp(self, n_epochs, batch_size, class_weight, validation_split=0.2):
    history = self.mlp_model.fit(
        self.X_train,
        self.y_train,
        batch_size = batch_size,
        epochs = n_epochs,
        shuffle = True,
        class_weight = class_weight,
        alidation_split=alidation_split
        verbose = 1)
    return history

  def evaluation(self, model, batch_size):
    prediction = model.predict(self.X_test, verbose=1, batch_size=batch_size)
    print(classification_report(self.y_test, prediction))
    plot_confusion_matrix(confusion_matrix(self.y_test, prediction))
    return prediction

  def data_process_lstm(self):
    lstm_X_train = self.X_train.reshape(self.X_train.shape[0], self.X_train.shape[-1], 1)
    self.lstm_X_test = self.X_test.reshape(self.X_test.shape[0], self.X_test.shape[-1], 1)

    self.train_dataset_sequence = tf.data.Dataset.from_tensor_slices((lstm_X_train , self.y_train))
    self.train_dataset_sequence = self.train_dataset_sequence.cache().shuffle(self.lstm_buffer_size).batch(self.lstm_batch_size).repeat()

    # self.test_dataset_sequence = tf.data.Dataset.from_tensor_slices((lstm_X_test , self.y_test))
    # self.test_dataset_sequence = self.test_dataset_sequence.cache().shuffle(self.lstm_buffer_size).batch(self.lstm_batch_size).repeat()

  def define_lstm_model(self, input_shape, output_shape=1):
    self.lstm_model = tf.keras.models.Sequential([tf.keras.layers.LSTM(units=32, return_sequences=True, input_shape=input_shape),
                                            tf.keras.layers.LSTM(units=64, return_sequences=True),
                                            tf.keras.layers.LSTM(units=32),
                                            tf.keras.layers.Dense(1,activation='sigmoid')])

    self.lstm_model.compile(loss = 'binary_crossentropy', metrics = ['accuracy'])

  def train_lstm(self):
    steps = int(np.ceil(self.X_train_sequence.shape[0] / self.lstm_batch_size))

    self.lstm_history = self.lstm_model.fit(
      self.train_dataset_sequence,
      epochs = self.lstm_epochs,
      steps_per_epoch = steps,
      # class_weight=class_weight,
      )

  def define_svm(self):
    self.svm_model = LinearSVC(C = 0.0001)
    self.svm_model.fit(self.X_train, self.y_train)

  def evaluate_svm(self):
    self.svm_prediction = self.svm_model.predict(self.X_test)
    print(classification_report(self.y_test, self.svm_prediction))
    plot_confusion_matrix(confusion_matrix(self.y_test, self.svm_prediction))

    def define_LGR(self):
    self.LGR_model = LogisticRegression()
    self.LGR_model.fit(self.X_train, self.y_train)

  def evaluate_LGR(self):
    self.LGR_prediction = self.LGR_model.predict(self.X_test)
    print(classification_report(self.y_test, self.LGR_prediction))
    plot_confusion_matrix(confusion_matrix(self.y_test, self.LGR_prediction))

  def define_knn(self):
    self.knn_model = KNeighborsClassifier(n_neighbors=4)
    self.knn_model.fit(self.X_train, self.y_train)

  def evaluate_knn(self):
    self.knn_prediction = self.knn_model.predict(self.X_test)
    print(classification_report(self.y_test, self.knn_prediction))
    plot_confusion_matrix(confusion_matrix(self.y_test, self.knn_prediction))

  def define_ensemble_concept(prediction1, prediction2):
    return np.logical_or(prediction1 , prediction2.T.ravel().round())

  def evaluate_ensemble_concept(self):
    print(classification_report(self.y_test, define_ensemble_concept(prediction1, prediction2)))
    plot_confusion_matrix(confusion_matrix(self.y_test, define_ensemble_concept(prediction1, prediction2)))

Prediction based on Ensemble Concept

Models' Summary

In [ ]:
conclusion = pd.DataFrame([['Neural Network',precision_1,recall_1,f1Score_1],
              ['Logistic Regression',0.53,0.17,0.26],
              ['SVM',0.38,0.01,0.03],
              ['KNN',0.94,0.56,0.70],
              ['Neural Network after Balancing',precision_2,recall_2,f1Score_2],
              ['Logistic Regression after Balancing',0.08,0.93,0.15],
              ['SVM after Balancing',0.06,0.97,0.11],
              ['KNN after Balancing',0.73,0.73,0.73],
              ['Ensemble concept after Balancing',0.65,0.81,0.72]],
              columns=["Algorithm","Precision","Recall","F1score"])
conclusion = conclusion.set_index('Algorithm')
conclusion.style.background_gradient(cmap="YlOrRd")


,Precision,Recall,F1score
Algorithm,,,
Neural Network,0.990000,0.340000,0.510000
Logistic Regression,0.530000,0.170000,0.260000
SVM,0.380000,0.010000,0.030000
KNN,0.940000,0.560000,0.700000
Neural Network after Balancing,0.880000,0.580000,0.700000
Logistic Regression after Balancing,0.080000,0.930000,0.150000
SVM after Balancing,0.060000,0.970000,0.110000
KNN after Balancing,0.730000,0.730000,0.730000
Ensemble concept after Balancing,0.650000,0.810000,0.720000


As you see After Blancing the dataset Ensemble concept and KNN model do the best !